In [171]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

FUNCTIONSSS

In [172]:
def ohe_prep(df, column, new_name): 
    ''' 
    Create One Hot Encoded features of a specific column
    ---
    Input: 
    df (pandas dataframe): Spotify Dataframe
    column (str): Column to be processed
    new_name (str): new column name to be used
        
    Output: 
    tf_df: One-hot encoded features 
    '''
    
    tf_df = pd.get_dummies(df[column])
    
    feature_names = tf_df.columns
    tf_df.columns = [new_name + "|" + str(i) for i in feature_names]
    tf_df.reset_index(drop = True, inplace = True)  
    return tf_df

def get_features_database(databaseDF):
    #Select Features
    databaseDF = databaseDF[["id","songName",
                "danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
                "valence","tempo","type"]]

    #OHE Features
    key_ohe = ohe_prep(databaseDF, 'key','key') * 0.5
    mode_ohe = ohe_prep(databaseDF, 'mode','mode') * 0.5

    ##Normalise/Scale Audio Columns
    float_cols = databaseDF.dtypes[databaseDF.dtypes == 'float64'].index.values
    floats = databaseDF[float_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    ##Combine all Features
    final = pd.concat([floats_scaled, key_ohe, mode_ohe, databaseDF["type"]], axis = 1)
    return final

def generate_rec(databaseDF, database_vector, user_vector):
    #Cosine Similarity
    databaseDF["sim"] = cosine_similarity(database_vector,user_vector)
    
    #Remove sim = 1 as it means its the same song
    databaseDF.drop(databaseDF[databaseDF['sim'] >= 1].index, inplace = True)

    rec_top5 = databaseDF.sort_values('sim',ascending = False).head(5)
    return rec_top5

Extract Both User and Dataset Csv

In [173]:
#Extract Database CSV
databaseDF = pd.read_csv("Dataset.csv", encoding="utf_8_sig")
databaseDF = databaseDF.drop(columns=['Unnamed: 0'])
#Selects columns that we want
databaseDF = databaseDF[["id","songName",
        "danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
        "valence","tempo",]]


In [174]:
#Extract User Playlist CSV
playlistDF = pd.read_csv("User_Playlist.csv", encoding="utf_8_sig")
playlistDF = playlistDF.drop(columns=['Unnamed: 0'])
#Reset Index
playlistDF = playlistDF.reset_index(drop = True)  
#Selects columns that we want
playlistDF = playlistDF[["id","songName",
        "danceability","energy","key","loudness","mode","speechiness","acousticness","instrumental","liveness",
        "valence","tempo",]] 



Normalise Features

In [175]:
#Merge user + dataset dataframe to normalise 
#Normalise takes min and max in dataframe as reference and change it to 0 and 1 respectively

#Group the dataframe as we gonna split it again alter
databaseDF["type"] = "Dataset"
playlistDF["type"] = "User"


#Check for duplicates in database
databaseDF = databaseDF.drop_duplicates(ignore_index= True)

#Merge the 2 datasets together
combinedDF = pd.concat([databaseDF,playlistDF], ignore_index=True)


##Normalise and get Vectors for Dataset + User
normalised_vector = get_features_database(combinedDF)

##Seperate User from databaseDF 
database_vector = normalised_vector[normalised_vector["type"] == "Dataset"]
user_vector = normalised_vector[normalised_vector["type"] == "User"]

#Drop "type" column
database_vector = database_vector.drop(columns="type")
user_vector = user_vector.drop(columns="type")
databaseDF = databaseDF.drop(columns=["type"])

Merge all User Vector to form single vector

In [176]:
##Single Vector Creation
final_user_vector_list = []
for i in user_vector.columns:
    final_user_vector_list.append(user_vector[i].sum()/len(user_vector[i]))

#Putting into a vector dataframe
final_user_vector = pd.DataFrame(columns=user_vector.columns,)
final_user_vector.loc[0] = final_user_vector_list



RECOMMEND TIME!

In [177]:
##Generate Recc Songs
result = generate_rec(databaseDF,database_vector,final_user_vector)
result

,id,songName,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumental,liveness,valence,tempo,sim
1429,6DIW7GJbuGZdHolRcPWprP,Roller Coaster,0.633,0.907,6,-1.459,0,0.0480,0.3170,0.0,0.2210,0.740,130.001,0.998225
311,2P4OICZRVAQcYAV2JReRfj,Waiting For Love,0.579,0.736,6,-3.863,0,0.0527,0.3100,0.0,0.1980,0.613,127.999,0.997994
151,4lIAPwAU6R8PAy2WhykC4i,#menow,0.703,0.909,6,-2.686,0,0.1020,0.1920,0.0,0.0717,0.830,117.989,0.997645
1414,2Oi0IO8K4BEbhPUdWcjNmv,NAVILLERA,0.642,0.957,6,-0.944,0,0.0500,0.0438,0.0,0.2730,0.801,125.008,0.997439
2159,18JosZY3HzD3lMy6iOOSAY,Superstar,0.673,0.592,6,-5.196,0,0.0537,0.1270,0.0,0.3360,0.705,142.222,0.997429
